This code goes from Topcoder's winners page, to the problem statement, to the problem detail of each SRM

In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'problem_statement'
number = 1
counter = 0
problemStatements = []
for Link in myLinks:
    URL = Link
    r = requests .get(URL)
    t = r.text

    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
    
        # Print the Problem Statement urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            problemStatements.append("http://community.topcoder.com" + s)


In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'round_overview'
number = 1
counter = 0
myLinks = []
while(1):
    URL = "http://community.topcoder.com/tc?module=SrmDivisionWins&sc=&sd=&nr=200&sr=" + str(number)
    number += 200
    r = requests .get(URL)
    t = r.text
    
    if ("An error has occurred when attempting to process your request" in t):
        print("There were no results matching your query.")
        print(counter)
        sys.exit()
    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
        
        if("topcoder" in s):
            continue;
        
        # Print the Round overview urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            myLinks.append("http://community.topcoder.com" + s)



This portion never would finish running on Jupyter, so I ran it on hydra instead. The only change I made was putting the links in a set to strip any duplicates

In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'ProblemDetail'
number = 1
counter = 0
problemDetails = []
for Statement in problemStatements:
    URL = Statement
    r = requests .get(URL)
    t = r.text

    soup = BeautifulSoup(t, 'html.parser')
    start = 0
    npages = 1
    for link in soup.findAll('a'):
        s = link.get('href')
    
        # Print the Problem Detail urls
        if(key in str(s)):
            counter += 1;
            #print ("http://community.topcoder.com" + s)
            problemDetails.append("http://community.topcoder.com" + s)

In [ ]:
import requests, json, sys, time, re
from bs4 import BeautifulSoup
key = 'ProblemDetail'
number = 1
counter = 0
#problemDetails = []

URL = "http://community.topcoder.com/tc?module=ProblemDetail&rd=15699&pm=12300"
r = requests .get(URL)
t = r.content
#print(t)